In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import copy
from sklearn import preprocessing
import json

In [18]:
df = pd.read_csv("movies_dataset.csv")


C:\Users\araqu\AppData\Local\Temp\ipykernel_11728\1893821005.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset.csv")


- MODIFICACION DE LA COLUMNA BELONG TO COLLECTION

In [19]:
import ast
import re

# definimos una expresión regular para extraer el nombre de la coleccion de la columna "belongs_to_collection"

pattern = r"'name': '([^']*)'"

# utilizamos la columna collection_name recbir el nombre de la coleccion utilizando el pattern
df["collection_name"] = df["belongs_to_collection"].str.extract(pattern, flags=re.IGNORECASE)

# el resultado se almacena en una nueva columna = collection_name

In [20]:
df[["belongs_to_collection","collection_name"]]

,belongs_to_collection,collection_name
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",Toy Story Collection
1,NaN,NaN
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",Grumpy Old Men Collection
3,NaN,NaN
4,"{'id': 96871, 'name': 'Father of the Bride Col...",Father of the Bride Collection
...,...,...
45461,NaN,NaN
45462,NaN,NaN
45463,NaN,NaN
45464,NaN,NaN


In [21]:
df.collection_name.nunique()

1644

In [22]:
# creamos una nueva columna para los registros duplicados
df["collection_individual"] = df["collection_name"].str.split(", ")
# duplicamos los registros con el metodo explode()
df = df.explode("collection_individual").reset_index(drop=True)
# Eliminamos la primer columna que creamos colecction_name
df.drop("collection_name", axis=1, inplace=True) 
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection


In [24]:
df.drop("belongs_to_collection", axis=1, inplace=True) 

- DESANIDACION DE LA COLUMNA SPOKEN LENGUAGES

In [26]:
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: eval(x) if pd.notnull(x) else x )
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: [lang['iso_639_1'] for lang in x] if isinstance(x,list) else x)
df['spoken_languages']

0            [en]
1        [en, fr]
2            [en]
3            [en]
4            [en]
           ...   
45537        [fa]
45538        [tl]
45539        [en]
45540          []
45541        [en]
Name: spoken_languages, Length: 45542, dtype: object

- MODIFICACION COLUMNAS GENRES

In [27]:
df['genres'] = df['genres'].apply( lambda x: eval(x) if pd.notnull(x) else x )
df['genres'] = df['genres'].apply( lambda x: [genero['name'] for genero in x] if isinstance(x,list) else x)
df['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45537                 [Drama, Family]
45538                         [Drama]
45539       [Action, Drama, Thriller]
45540                              []
45541                              []
Name: genres, Length: 45542, dtype: object

- MODIFICACION DE LA COLUMNA PRODUCTION COMPANIES

In [28]:
df['production_companies'] = df['production_companies'].apply( lambda x: eval(x) if pd.notnull(x) else x )
df['production_companies'] = df['production_companies'].apply( lambda x: [production['name'] for production in x] if isinstance(x,list) else x)
df['production_companies']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45537                                                   []
45538                                        [Sine Olivia]
45539                            [American World Pictures]
45540                                          [Yermoliev]
45541                                                   []
Name: production_companies, Length: 45542, dtype: object

- MODIFICACION DE LA COLUMNA PRODUCTION COUNTRIES

In [29]:
df['production_countries'] = df['production_countries'].apply( lambda x: eval(x) if pd.notnull(x) else x )
df['production_countries'] = df['production_countries'].apply( lambda x: [country['name'] for country in x] if isinstance(x,list) else x)
df['production_countries']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45537                        [Iran]
45538                 [Philippines]
45539    [United States of America]
45540                      [Russia]
45541              [United Kingdom]
Name: production_countries, Length: 45542, dtype: object

- EN EL PROCESO DE REVISION DE REVENUE Y BUDGET SE ENCUENTRAN TRES VALORES ERRONEOS, SE ELIMINAN PARA EVITAR FUTUROS ERRORES

In [31]:
df['budget'] = df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', value= 0)
df['budget'] = df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', value= 0)
df['budget'] = df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', value= 0)

- CALCULO DE LA COLUMNA RETURN

In [32]:
df['budget'] = df['budget'].replace('N/A', value= 0)
df['revenue'] = df['revenue'].fillna(value=0)
df['budget'] = df['budget'].fillna(value=0)
df[['revenue', 'budget']] = df[['revenue', 'budget']].astype(float)

In [33]:
def calcular_retorno(row):
    revenue = row['revenue']
    budget = row['budget']
    
    # Verificar si hay datos faltantes o el presupuesto es igual a cero
    if pd.isnull(revenue) or pd.isnull(budget) or budget == 0:
        return 0
    
    # Calcular el retorno de inversión y redondearlo a dos decimales
    return round(revenue / budget, 2)

# Aplicar la función a cada fila del DataFrame para crear la columna "return"
df['return'] = df.apply(calcular_retorno, axis=1)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45542 entries, 0 to 45541
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45542 non-null  object 
 1   budget                 45542 non-null  float64
 2   genres                 45542 non-null  object 
 3   homepage               7805 non-null   object 
 4   id                     45542 non-null  object 
 5   imdb_id                45525 non-null  object 
 6   original_language      45531 non-null  object 
 7   original_title         45542 non-null  object 
 8   overview               44588 non-null  object 
 9   popularity             45537 non-null  object 
 10  poster_path            45156 non-null  object 
 11  production_companies   45539 non-null  object 
 12  production_countries   45539 non-null  object 
 13  release_date           45455 non-null  object 
 14  revenue                45542 non-null  float64
 15  ru

- MODIFICACION DE LA COLUMNA RELEASE DATE

In [37]:
# Primera parte es comprobar los datos y convertir los que no formato de datetime en nulos
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
# nos aseguramos que todos los registros tengan el mismo formato de fecha AAAA-MM-DD
df["release_date"] = pd.to_datetime(df["release_date"]).dt.strftime("%Y-%m-%d") 

In [38]:
#identificamos la cantidad de nulos de la columna "release_date"
df["release_date"].isnull().sum()

90

In [39]:
#eliminamos los registros con nulos
df = df.dropna(subset = ["release_date"])
# Verificamos nuevamente que no existan nulos.
df["release_date"].isnull().sum()

0

In [40]:
# creamos la columna "release_year" que va a contener solo el año de la columna "release_date"
df["release_date"] = pd.to_datetime(df["release_date"])
# Extraemos los valores para que en release year quede el año de cada pelicula
df["release_year"]=df["release_date"].dt.year
# verificamos como quedaron las columnas
df[["release_date","release_year"]].tail(5)

,release_date,release_year
45536,1991-05-13,1991
45538,2011-11-17,2011
45539,2003-08-01,2003
45540,1917-10-21,1917
45541,2017-06-09,2017


In [41]:
# Extraemos los valores para que en release month quede el mes de cada pelicula
df["release_month"]=df["release_date"].dt.month
# Extraemos los valores para que en release day quede el día de cada pelicula
df["release_day"]=df["release_date"].dt.day_of_week

- SE ELIMINAN LAS COLUMNAS 'belongs_to_collection','video','imdb_id','adult','original_title','poster_path','homepage'COMO LO ESPECIFICA EL PROCESO

In [44]:
df.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,tagline,title,video,vote_average,vote_count,collection_individual,return,release_year,release_month,release_day
0,False,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,12.45,1995,10,0
1,False,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,4.04,1995,12,4


In [45]:
columnas = ['video','imdb_id','adult','original_title','poster_path','homepage']
df.drop(columns= columnas, inplace=True)

In [47]:
df.to_csv("movies_dataset_Limpio.csv",index=False)

In [46]:
df.head(2)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,status,tagline,title,vote_average,vote_count,collection_individual,return,release_year,release_month,release_day
0,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,...,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,12.45,1995,10,0
1,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,4.04,1995,12,4
